In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras

In [ ]:
import os
import zipfile

local_zip = '/content/drive/MyDrive/Capstone/Project/Data/C22-PS300.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('./data')
zip_ref.close()

In [ ]:
base_dir = './data/C22-PS300'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')

# Directory with our training acne pictures
train_blackhead_dir = os.path.join(train_dir, 'blackhead')
train_folikulitis_dir = os.path.join(train_dir, 'folikulitis')
train_melasma_dir = os.path.join(train_dir, 'melasma')
train_nodules_dir = os.path.join(train_dir, 'nodules')
train_papula_dir = os.path.join(train_dir, 'papula')
train_pustula_dir = os.path.join(train_dir, 'pustula')
train_rosacea_dir = os.path.join(train_dir, 'rosacea')
train_whitehead_dir = os.path.join(train_dir, 'whitehead')
train_normalface_dir = os.path.join(train_dir, 'normalface')

# Directory with our validation acne pictures
validation_blackhead_dir = os.path.join(validation_dir, 'blackhead')
validation_folikulitis_dir = os.path.join(validation_dir, 'folikulitis')
validation_melasma_dir = os.path.join(validation_dir, 'melasma')
validation_nodules_dir = os.path.join(validation_dir, 'nodules')
validation_papula_dir = os.path.join(validation_dir, 'papula')
validation_pustula_dir = os.path.join(validation_dir, 'pustula')
validation_rosacea_dir = os.path.join(validation_dir, 'rosacea')
validation_whitehead_dir = os.path.join(validation_dir, 'whitehead')
validation_normalface_dir = os.path.join(validation_dir, 'normalface')

In [ ]:
train_blackhead_fnames = os.listdir(train_blackhead_dir)
train_folikulitis_fnames = os.listdir(train_folikulitis_dir)
train_melasma_fnames = os.listdir(train_melasma_dir)
train_nodules_fnames = os.listdir(train_nodules_dir)
train_papula_fnames = os.listdir(train_papula_dir)
train_pustula_fnames = os.listdir(train_pustula_dir)
train_rosacea_fnames = os.listdir(train_rosacea_dir)
train_whitehead_fnames = os.listdir(train_whitehead_dir)
train_normalface_fnames = os.listdir(train_normalface_dir)

print(train_blackhead_fnames[:3])

In [ ]:
print('total training blackhead images :', len(os.listdir(train_blackhead_dir)))
print('total training folikulitis images :', len(os.listdir(train_folikulitis_dir)))
print('total training melasma images :', len(os.listdir(train_melasma_dir)))
print('total training nodules images :', len(os.listdir(train_nodules_dir)))
print('total training papula images :', len(os.listdir(train_papula_dir)))
print('total training pustula images :', len(os.listdir(train_pustula_dir)))
print('total training rosacea images :', len(os.listdir(train_rosacea_dir)))
print('total training whitehead images :', len(os.listdir(train_whitehead_dir)))
print('total training normalface images :', len(os.listdir(train_normalface_dir)))

print('total validation blackhead images :', len(os.listdir(validation_blackhead_dir)))
print('total validation folikulitis images :', len(os.listdir(validation_folikulitis_dir)))
print('total validation melasma images :', len(os.listdir(validation_melasma_dir)))
print('total validation nodules images :', len(os.listdir(validation_nodules_dir)))
print('total validation papula images :', len(os.listdir(validation_papula_dir)))
print('total validation pustula images :', len(os.listdir(validation_pustula_dir)))
print('total validation rosacea images :', len(os.listdir(validation_rosacea_dir)))
print('total validation whitehead images :', len(os.listdir(validation_whitehead_dir)))
print('total validation normalface images :', len(os.listdir(validation_normalface_dir)))

In [ ]:
model = keras.applications.mobilenet.MobileNet()

In [ ]:
model.summary()

In [ ]:
model_mod = model.layers[-1].output
x = keras.layers.Flatten()(model_mod)
denser = keras.layers.Dense(128, activation='relu')(x)
DO = keras.layers.Dropout(0.5)(denser)
prediction = keras.layers.Dense(9, activation='softmax')(DO)

In [ ]:
fullmodel = keras.Model(inputs=model.input, outputs=prediction)

In [ ]:
fullmodel.summary()

In [ ]:
# from tensorflow.keras.optimizers import Adam
# optimizer = Adam(learning_rate=0.001)
fullmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
for layers in model.layers[:]:
  layers.trainable=False

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator(
      rescale = 1.0/255.,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      rotation_range=40,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
test_datagen  = ImageDataGenerator(
      rescale = 1.0/255.)

# --------------------
# Flow training images in batches of 32 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=27,
                                                    class_mode='categorical',
                                                    target_size=(224, 224))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=22,
                                                         class_mode  = 'categorical',
                                                         target_size = (224, 224))


In [ ]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('val_accuracy')>0.72):
      print('Good enough accuracy so cancelling training!')
      self.model.stop_training = True

callback = MyCallback()

In [ ]:
history = fullmodel.fit(train_generator,
                        validation_data=validation_generator,
                        steps_per_epoch=23,
                        epochs=300,
                        validation_steps=11,
                        verbose=2,
                        callbacks=[callback])

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc, label='Training')
plt.plot  ( epochs, val_acc, label='Validation')
plt.title ('Training and validation accuracy')
plt.legend()
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss, label='Training')
plt.plot  ( epochs, val_loss, label='Validation')
plt.legend()
plt.title ('Training and validation loss')

In [ ]:
from google.colab import files
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import numpy as np
class_dict = {v : k for k, v in train_generator.class_indices.items()}
uploads = files.upload()

n_files = len(uploads)
fig = plt.figure(figsize=(2, 2 * n_files))

for i, filename in enumerate(uploads):
  img = image.load_img(
      filename,
      target_size=(224, 224)
  )
  x = image.img_to_array(img)/255
  x = np.expand_dims(x, axis=0)

  predict = fullmodel.predict(x)
  class_prediction = np.argmax(predict)
  prediction = class_dict[class_prediction]
  
  
  ax = fig.add_subplot(n_files, 1, i+1)
  ax.imshow(img)
  ax.set_title(prediction)
  ax.axis(False)
  print('percentage:')
  print(predict)


In [ ]:
#h5 model
saved_model_path = "/content/drive/MyDrive/Capstone/Project/Model/Model.h5"

model.save(saved_model_path)

In [ ]:
!sudo pip install tensorflowjs

In [ ]:
#TensorFlow.js model
!tensorflowjs_converter --input_format=keras "/content/drive/MyDrive/Capstone/Project/Model/Model.h5" /content/drive/MyDrive/Capstone/Project/Model